In [1]:
#| default_exp classes.DomoPDP

In [2]:
#| exporti
import asyncio
import datetime as dt
import io
import json
from dataclasses import dataclass, field
from enum import Enum, auto

from fastcore.basics import patch_to

import httpx
import pandas as pd

#from ..utils.chunk_execution import chunk_list
#from ..utils.DictDot import DictDot
import domolibrary.utils.DictDot as util_dd
#from . import DomoCertification as dmdc
#from .DomoAuth import DomoDeveloperAuth, DomoFullAuth
import domolibrary.client.DomoAuth as dmda
#from .routes import pdp_routes
import domolibrary.routes.pdp as pdp_routes
import domolibrary.client.DomoError as de
import domolibrary.classes.DomoDataset as dmd



In [3]:
#| export
@dataclass
class PDP_Policy():
    dataset_id: str
    filter_group_id: str
    name: str
    resources: list
    parameters: list

    @classmethod
    def _from_json(cls, obj):
        dd = util_dd.DictDot(obj)

        return cls(dataset_id=dd.dataSourceId,
                   filter_group_id=dd.filterGroupId,
                   name=dd.name,
                   resources=dd.resources,
                   parameters=dd.parameters)

    @staticmethod
    def generate_parameter_simple(column_name, column_values_list, operator='EQUALS', ignore_case: bool = True):
        return pdp_routes.generate_policy_parameter_simple(column_name=column_name,
                                                           column_values_list=column_values_list,
                                                           operator=operator,
                                                           ignore_case=ignore_case)

    @staticmethod
    def generate_body(policy_name, dataset_id, parameters_list, policy_id=None, user_ids=None, group_ids=None, virtual_user_ids=None):
        return pdp_routes.generate_policy_body(policy_name=policy_name,
                                               dataset_id=dataset_id,
                                               parameters_list=parameters_list,
                                               policy_id=policy_id,
                                               user_ids=user_ids,
                                               group_ids=group_ids,
                                               virtual_user_ids=virtual_user_ids)

    @classmethod
    async def upsert_policy(cls, 
                            auth: dmda.DomoAuth,
                            dataset_id: str,
                            policy_definition: dict, # body sent to the API (uses camelCase instead of snake_case)
                            debug_api: bool = False,
                            debug_prn: bool = False
                            ):
        
        # print(policy_definition)
        
        if policy_definition.get('filterGroupId'):
            if debug_prn:
                print(f'Updating policy in {auth.domo_instance}')    
            res = await pdp_routes.update_policy(auth=auth,
                                                 dataset_id=dataset_id,
                                                 filter_group_id=policy_definition.get(
                                                     'filterGroupId'),
                                                 body=policy_definition,
                                                 debug_api=debug_api)
            return cls._from_json(res.response)
        else:
            if debug_prn:
                print(f'Policy does not exist. Creating policy in {auth.domo_instance}')
            res = await pdp_routes.create_policy(auth=auth,
                                                 dataset_id=dataset_id,
                                                 body=policy_definition,
                                                 debug_api=debug_api)

            return cls._from_json(res.response)
    

In [4]:
#| export
class Dataset_PDP_Policies:

    dataset: dict  # domo dataset class
    policies: list[PDP_Policy] = None  

    def __init__(self, dataset):
        self.dataset = dataset
        self.policies = []

    async def get_policies(self, 
                           auth: dmda.DomoAuth = None, 
                           dataset_id: str = None, 
                           debug_api: bool = False):

        dataset_id = dataset_id or self.dataset.id
        auth = auth or self.dataset.auth

        res = await pdp_routes.get_pdp_policies(auth=auth, dataset_id=dataset_id, debug_api=debug_api)


        if res.status == 200:
            domo_policy = [PDP_Policy._from_json(
                policy_obj) for policy_obj in res.response]
            self.policies = domo_policy
            return domo_policy

### sample implementation of get_policies

In [5]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]
dataset_pdp_policies = Dataset_PDP_Policies(dataset=dmd.DomoDataset(auth=token_auth, id=dataset_id))
await dataset_pdp_policies.get_policies()


[PDP_Policy(dataset_id='04c1574e-c8be-4721-9846-c6ffa491144b', filter_group_id=1149, name='Test Update Policy Name', resources=DictDot(USER=['1893952720']), parameters=[DictDot(name='objectID', value='000003007', values=['000003007'], type='COLUMN', operator='EQUALS', not=False, ignoreCase=True)])]

In [6]:
# sample_parameter = dataset_pdp_policies.policies.generate_policy_parameter_simple(
#     column_name="Employee ID",
#     column_values_ls=['456'],
#     operator="EQUALS",
#     ignore_case=True,
# )

# body = dataset_pdp_policies.policies.generate_policy_body(
#     policy_name="Test Policy Name Employee 4567",
#     dataset_id = dataset_id,
#     policy_id=None,  # including the policy_id updates that existing policy
#     parameters_ls=sample_parameter,
#     user_ids=['1893952720']
# )

In [7]:
#await PDP_Policy.upsert_policy()

### Search PDP Policies by name and ID

In [8]:
#| export
class SearchPDP_NotFound(de.DomoError):
    def __init__(self, 
                 domo_instance,
                 dataset_id,
                 message='not found',
                 function_name='search_pdp'):

        super().__init__(domo_instance=domo_instance, entity_id=dataset_id, message=message, function_name=function_name)
         
@patch_to(Dataset_PDP_Policies, cls_method=True)
async def search_pdp_policies(
    cls: Dataset_PDP_Policies,
    auth: dmda.DomoAuth,
    search: str,
    dataset_id: str = None,
    search_method: str = 'id' or 'name',
    is_exact_match: bool = True,
    return_raw: bool = False, 
    debug_api: bool = False,
    session: httpx.AsyncClient = None
):
    
    all_pdp_policies = await Dataset_PDP_Policies(cls).get_policies(
        auth = auth,
        dataset_id = dataset_id,
        debug_api=debug_api
    )
    
    if return_raw:
        return all_pdp_polcies

    if search_method == 'name':
        if is_exact_match:
            policy_search = next((policy for policy in all_pdp_policies if policy.name == search), None)
            #print(policy_search)   
            
            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy named "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance
                )  
            
            return policy_search
        else:
            policy_search = [policy for policy in all_pdp_policies if search.lower() in policy.name.lower()]
            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy name containing "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance
                )  
            
            return policy_search
    else:
        policy_search = next((policy for policy in all_pdp_policies if policy.filter_group_id == search), None)
         
    if not policy_search:
        raise SearchPDP_NotFound(
            dataset_id=dataset_id,
            message=f'There is no policy id "{search}" on dataset_id {dataset_id}',
            domo_instance=auth.domo_instance
        )  
          
    return policy_search    
    

### sample implementation of search_pdp_policies

In [9]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

#authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

#instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

#instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

#pull all policies for that dataset
await dpp.get_policies()

#search policies based on name
try:
    policy = await dpp.search_pdp_policies(
        auth=token_auth, 
        dataset_id=dataset.id, 
        search='Test Policy Name Employee 123', 
        search_method='name', 
        is_exact_match=True, 
        debug_api=False
    )
    policy

except SearchPDP_NotFound as e:
    print(e)

search_pdp: 04c1574e-c8be-4721-9846-c6ffa491144b :There is no policy named "Test Policy Name Employee 123" on dataset_id 04c1574e-c8be-4721-9846-c6ffa491144b at domo-community


### Enable/Disable PDP on dataset

In [10]:
#| export
@patch_to(Dataset_PDP_Policies, cls_method=True)
async def toggle_dataset_pdp(
    cls: Dataset_PDP_Policies,
    auth: dmda.DomoAuth,
    dataset_id: str = None,
    is_enable: bool = True, # True will enable pdp, False will disable pdp
    debug_api: bool = False,
    session: httpx.AsyncClient = None
):

    return await pdp_routes.toggle_pdp(
        auth=auth,
        dataset_id=dataset_id,
        is_enable=is_enable,
        debug_api=debug_api,
        session=session
    )



### sample implementation of toggle_dataset_pdp

In [11]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

#authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

#instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

#instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

#toggle pdp on dataset
await dpp.toggle_dataset_pdp(auth=token_auth, dataset_id=dataset.id, is_enable=True, debug_api=False)

ResponseGetData(status=200, response={'enabled': True, 'secured': False, 'external': False}, is_success=True)